In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Clean text data

df = pd.read_csv('/content/spam_ham_dataset.csv')
df['clean_text'] = df['text'].str.lower()
df['clean_text'] = df['clean_text'].str.replace(r'[^\w\s]+', '')
df['clean_text'] = df['clean_text'].str.replace(r'\s+', ' ')
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['clean_text']).toarray()
y = df['label_num'].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [2]:
import numpy as np

class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)

class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])

        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples

class Activation_Softmax_Loss_CategoricalCrossentropy():
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def forward(self, inputs, y_true):
        self.activation.forward(inputs)
        self.output = self.activation.output
        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)

        self.dinputs = dvalues.copy()
        self.dinputs[range(samples), y_true] -= 1
        self.dinputs = self.dinputs / samples

In [3]:
# Input layer → Hidden layer (ReLU) → Output layer (Softmax)
dense1 = Layer_Dense(n_inputs=5000, n_neurons=64)
activation1 = Activation_ReLU()

dense2 = Layer_Dense(n_inputs=64, n_neurons=5)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

In [4]:
# Training parameters
epochs = 1000
learning_rate = 0.1

for epoch in range(epochs):
    # Forward pass
    dense1.forward(X_train)
    activation1.forward(dense1.output)

    dense2.forward(activation1.output)
    loss = loss_activation.forward(dense2.output, y_train)

    # Calculate accuracy
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y_train.shape) == 2:
        y_train_labels = np.argmax(y_train, axis=1)
    else:
        y_train_labels = y_train
    accuracy = np.mean(predictions == y_train_labels)

    # Backward pass
    loss_activation.backward(loss_activation.output, y_train)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    dense1.weights -= learning_rate * dense1.dweights
    dense1.biases -= learning_rate * dense1.dbiases
    dense2.weights -= learning_rate * dense2.dweights
    dense2.biases -= learning_rate * dense2.dbiases

    # Print loss and accuracy
    if epoch % 100 == 0 or epoch == epochs - 1:
        print(f"Epoch {epoch}: Loss={loss:.4f}, Accuracy={accuracy:.4f}")

Epoch 0: Loss=1.6092, Accuracy=0.2855
Epoch 100: Loss=0.7277, Accuracy=0.7084
Epoch 200: Loss=0.6503, Accuracy=0.7084
Epoch 300: Loss=0.6197, Accuracy=0.7084
Epoch 400: Loss=0.5917, Accuracy=0.7084
Epoch 500: Loss=0.5374, Accuracy=0.7084
Epoch 600: Loss=0.4356, Accuracy=0.7244
Epoch 700: Loss=0.3215, Accuracy=0.9139
Epoch 800: Loss=0.2392, Accuracy=0.9628
Epoch 900: Loss=0.1868, Accuracy=0.9700
Epoch 999: Loss=0.1524, Accuracy=0.9768


In [5]:
# Forward pass on test set
dense1.forward(X_test)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
loss = loss_activation.forward(dense2.output, y_test)

# Calculate test accuracy
test_predictions = np.argmax(loss_activation.output, axis=1)
test_accuracy = np.mean(test_predictions == y_test)
print(f"\nTest Accuracy: {test_accuracy:.4f}")


Test Accuracy: 0.9710


In [6]:
import pandas as pd
import numpy as np

def predict_spam_from_file(file_path):
    # Read the email text from the file clean the text, vectorize it and make a prediction

    with open(file_path, 'r', encoding='utf-8') as f:
        email_text = f.read()
    s = pd.Series([email_text])

    clean_text = (
        s.str.lower()
         .str.replace(r'[^\w\s]+', '', regex=True)
         .str.replace(r'\s+', ' ', regex=True)
         .str.strip()
         .iloc[0]
    )

    X_new = vectorizer.transform([clean_text]).toarray()

    dense1.forward(X_new)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss_activation.forward(dense2.output, np.array([0]))

    prob_spam = loss_activation.output[0][1]
    return "spam" if prob_spam > 0.5 else "not spam", prob_spam

# Example usage:
file_path = "/content/email_ham1_real.txt"
prediction, confidence = predict_spam_from_file(file_path)
print(f"Prediction: {prediction} (Probability of spam: {confidence:.2f})\n")
file_path = "/content/email_ham2_real.txt"
prediction, confidence = predict_spam_from_file(file_path)
print(f"Prediction: {prediction} (Probability of spam: {confidence:.2f})\n")
file_path = "/content/email_spam1_real.txt"
prediction, confidence = predict_spam_from_file(file_path)
print(f"Prediction: {prediction} (Probability of spam: {confidence:.2f})\n")
file_path = "/content/email_spam2_real.txt"
prediction, confidence = predict_spam_from_file(file_path)
print(f"Prediction: {prediction} (Probability of spam: {confidence:.2f})\n")
file_path = "/content/email_spam3.txt"
prediction, confidence = predict_spam_from_file(file_path)
print(f"Prediction: {prediction} (Probability of spam: {confidence:.2f})\n")

Prediction: not spam (Probability of spam: 0.19)

Prediction: not spam (Probability of spam: 0.13)

Prediction: spam (Probability of spam: 0.87)

Prediction: spam (Probability of spam: 0.91)

Prediction: spam (Probability of spam: 0.74)

